In [2]:
from confluent_kafka.admin import AdminClient, NewTopic


In [15]:
admin_client = AdminClient({
    "bootstrap.servers": "kafka-stream-processing:9092"
})

In [19]:
topic_list = []
topic_list.append(NewTopic("example_tasdopic", 1, 1))

In [20]:
admin_client.create_topics(topic_list)


{'example_tasdopic': <Future at 0x7fe591c18310 state=running>}

In [21]:
topic_list

[NewTopic(topic=example_tasdopic,num_partitions=1)]